In [ ]:
from approxposterior import approx, gpUtils, likelihood as lh, utility as ut
import numpy as np

# Define algorithm parameters
training_set_size = 50                           # Initial size of training set
new_points_per_iteration = 20                            # Number of new points to find each iteration
max_iterations = 5                        # Maximum number of iterations
prior_bounds = [(-5,5), (-5,5)]         # Prior bounds
algorithm = "bape"                # Use the Kandasamy et al. (2017) formalism
seed = 57                         # RNG seed
np.random.seed(seed)

# emcee MCMC parameters
samplerKwargs = {"nwalkers" : 20}        # emcee.EnsembleSampler parameters
mcmcKwargs = {"iterations" : int(2.0e4)} # emcee.EnsembleSampler.run_mcmc parameters

# Sample design points from prior
theta = lh.rosenbrockSample(training_set_size)

# Evaluate forward model log likelihood + lnprior for each theta
y = np.zeros(len(theta))
for ii in range(len(theta)):
    y[ii] = lh.rosenbrockLnlike(theta[ii]) + lh.rosenbrockLnprior(theta[ii])

# Default GP with an ExpSquaredKernel
gp = gpUtils.defaultGP(theta, y, white_noise=-12)

# Initialize object using the Wang & Li (2018) Rosenbrock function example
approximate_posterior = approx.ApproxPosterior(theta=theta,
                            y=y,
                            gp=gp,
                            lnprior=lh.rosenbrockLnprior,
                            lnlike=lh.rosenbrockLnlike,
                            priorSample=lh.rosenbrockSample,
                            bounds=prior_bounds,
                            algorithm=algorithm)

# Run!
approximate_posterior.run(m=new_points_per_iteration, nmax=max_iterations, estBurnin=True, nGPRestarts=3, mcmcKwargs=mcmcKwargs,
       cache=False, samplerKwargs=samplerKwargs, verbose=True, thinChains=False,
       onlyLastMCMC=True)

In [ ]:
# Check out the final posterior distribution!
import corner

# Load in chain from last iteration
samples = approximate_posterior.sampler.get_chain(discard=approximate_posterior.iburns[-1], flat=True, thin=approximate_posterior.ithins[-1])

# Corner plot!
fig = corner.corner(samples, quantiles=[0.16, 0.5, 0.84], show_titles=True,
                    scale_hist=True, plot_contours=True)

# Plot where forward model was evaluated - uncomment to plot!
fig.axes[2].scatter(approximate_posterior.theta[training_set_size:,0], approximate_posterior.theta[training_set_size:,1], s=10, color="red", zorder=20)

# Save figure
fig.savefig("finalPosterior.png", bbox_inches="tight")

In [2]:
import numpy as np

In [3]:
np.random.seed(57)

# Define the true bias of the coin
true_bias = 0.6

# Decide on the number of coin flips
n_flips = 1000

# Generate synthetic coin flips
synthetic_data = np.random.choice([0, 1], size=n_flips, p=[1-true_bias, true_bias])

n_heads = np.sum(synthetic_data)
n_tails = n_flips - n_heads

print("Number of heads: ", n_heads)
print("Number of tails: ", n_tails)

Number of heads:  594
Number of tails:  406
